In [ ]:
#in this script, we implement our algorithm for matrix learning, following the proof
#of the theorem... of ... 

In [1]:
using JuMP 
using Mosek

<h1>Implementancion de las funciones<h1>

In [ ]:

# @param:weights : weights of the observed matrices. Sum of weights must be 1. and non-negative. the size is equal to the amount
#of observed graphs.
#@param: delta : size of the ball of the Wasserstein metric. delta > 0.
#@param: A : cluster form of the data. A entries must be 0 or 1. 
#@param: B : vector of matrices containing the observations of the graph B. If X,Y are matrices, an array of matrices
#is of the form Array[X,Y].
#@param: n : the dimension of the matrices.
#@param:N : number of observations.
#The set over which the is problem is optimized is:

In [ ]:
display("text/latex",""" El conjunto sobre el que optimizamos es:
\$ O = \\{\\lambda,S_1,..,S_N,Z^{1},..,Z^{N},C^{1},...,C^{N},W,W_1,W_2 \\}\\subseteq \\mathbb{R}\\times \\mathbb{R}^N
\\times Sim^2(\\mathbb{R}^n)^N \\times Sim^2(\\mathbb{R}^n)^N \\times Sim^2(\\mathbb{R}^)^3
\$""")

In [22]:
#Computes the error of estimating the cluster structure of the observations by the matrix A.
function learnError(weights,delta,A,B,n,N)
    one =ones(n)
    m = Model(solver=MosekSolver())
    @variable(m,lambda>=0)
     @variable(m,S[1:N])
    @variable(m, W[1:n,1:n], Symmetric)
    Z = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
    C = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
    @variable(m,W1[1:n,1:n])
    @variable(m,W2[1:n,1:n])
    #Constraints for Z
    [@constraint(m, -Z[p][k,l] <= A[k,l]-B[p][k,l]) for p in 1:N for k in 1:n for l in 1:n]
    [@constraint(m,  A[k,l]-B[p][k,l]<=Z[p][k,l]) for p in 1:N for k in 1:n for l in 1:n]
    #Constraints for C 
    [@constraint(m,C[p][k,l]>=0) for p in 1:N for k in 1:n for l in 1:n]
    [@constraint(m, prod(C[p],n)<=S[p]-trace((2*A-one*one'-W)'*B[p])) for p in 1:N]
    [@constraint(m,-C[p][k,l]<=2*A[k,l]-1-W[k,l]) for p in 1:N for k in 1:n for l in 1:n]
    #Constraints for W
    @SDconstraint(m,[W1 W;W W2]>=0)
    @constraint(m,trace(W1)+trace(W2)<=2*lambda)
    #Objective function, calls the function prod
    @objective(m,Min,lambda*delta+ dot(weights,S+prod.(Z,n)))
    status = solve(m)
    loss= getobjectivevalue(m)
    #return(loss)
    return([getvalue(C),getvalue(W),getvalue(S)])
end


learnError (generic function with 1 method)

In [2]:
#Finds the matrix A which minimices the expected error of cluster structure. This
#time, A is a simmtric entrywise-positive matrix.
#The parameters are the same as in learnError function
function learnGraph(weights,delta,B,n,N)
    one =ones(n)
    m = Model(solver=MosekSolver())
    @variable(m,lambda>=0)
     @variable(m,S[1:N])
    @variable(m, W[1:n,1:n], Symmetric)
    @variable(m,A[1:n,1:n], Symmetric)
    Z = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
    C = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
    @variable(m,W1[1:n,1:n])
    @variable(m,W2[1:n,1:n])
    #Constraints for Z
    [@constraint(m, -Z[p][k,l] <= A[k,l]-B[p][k,l]) for p in 1:N for k in 1:n for l in 1:n]
    [@constraint(m,  A[k,l]-B[p][k,l]<=Z[p][k,l]) for p in 1:N for k in 1:n for l in 1:n]
    #Constraints for C 
    [@constraint(m,C[p][k,l]>=0)for p in 1:N for k in 1:n for l in 1:n]
    [@constraint(m, prod(C[p],n)<=S[p]-trace((2*A-one*one'-W)'*B[p])) for p in 1:N]
    [@constraint(m,-C[p][k,l]<=2*A[k,l]-1-W[k,l])for p in 1:N for k in 1:n for l in 1:n]
    #Constraints for W
    @SDconstraint(m,[W1 W;W W2]>=0)
    @constraint(m,trace(W1)+trace(W2)<=2*lambda)
    #constrains for A
    [@constraint(m,A[k,l]>=0) for k in 1:n for l in 1:n]
    [@constraint(m,A[k,l]<=1) for k in 1:n for l in 1:n]
    #Objective function, calls the function prod
    @objective(m,Min,lambda*delta+ dot(weights,S+prod.(Z,n)))
    status = solve(m)
    return([roundZeroMatrix(getvalue(A)),getvalue(S),getvalue(lambda),roundZeroMatrix(getvalue(W)),getobjectivevalue(m)])
end

learnGraph (generic function with 1 method)

<h1> Funciones auxiliares <h1>

In [3]:
#Aux functions
#def function prod computes the product 1^t*Z*t.
#@param: n : the dimension of the matrices.
function prod(Z,n)
    return(ones(n)'*Z*ones(n))
end

#def function roundMatrix applies the function roundZero to every element of the matrix.
#@param: A : matrix to round
function roundZeroMatrix(A)
    map(A) do x
        roundZero(x)
    end
end

#def function roundZero rounds close reals to zero to zero
function roundZero(a)
    if(abs(a)<=0.0001)
        return(0)
    end
    return(a) 
end

#def function nuclearNorm Computes the nuclear norm of a matrix
#@param mat : mat to compute the nuclear norm
function nuclearNorm(mat)
    return(sum(svdfact(mat)[:S]))
end

#function which computes the estimation of the error as by theorem 3.1
#@param delta: radius of the wasserstein metric ball.
#@param A : solution of the optimization problem.
#@param B : Array of the observation matrix
#@param N : numero de observaciones
#@param n : dimension de las matrices
function estimarCota(delta,A,B,N,n)
    a = delta*nuclearNorm(2*A-ones(n)*ones(n)')
    [B[i] = A-B[i] for i in 1:N]
    d = (1/N)*sum(norm1.(B))
    return(a+d)
end
#function that computes the norm1 of a matrix. note that the norm(,1) implemented
#in julia is the induced 1 norm.
#@param W: a matrix
function norm1(W)
    a= 0
    map(W) do x
        a = abs(x)+a
    end
    return(a)
end

#function to generate a random Erdos-renyi graph with a given number of 
# clusters and probabilities. Creates an inferior triangluar matrix and then sums its transpose.
#@param: probabilities: vector of probabilities>0.5 of each intracluster probability.
#The size of the vector is the number of clusters. min(probabilities)>1/2
#@param: sizes : the sizes of the clusters.
#@param: n : the dimension of the adjacency matrix.
#@param: q : outercluster adjacency probability : q<1/2
#@Pre: the sum of the sizes must be n.
function erdosgraph(probabilities,q,sizes,n)
    A= zeros(n,n)
    contador2=0
    proba = 1
    contador1=1
    for k in sizes
        contador2=contador2+k
        for j in contador1:contador2
            for i in j+1:n
                rand1=rand()
                rand2=rand()
                if(i>=contador1 && i<=contador2)
                    if(rand1<=probabilities[proba])
                        A[i,j]=1
                    end
                else
                    if(rand2<=q)
                        A[i,j]=1
                    end
                end
            end
        end
        proba=proba+1
        contador1=contador1+k
    end
    return(A+A')
end

erdosgraph (generic function with 1 method)

In [6]:
#Same as erdosgraph but amount of clusters is kept to 2.
function erdosgraph2(probabilities,q,sizes,n)
    A= zeros(n,n)
    contador2=0
    proba = 1
    contador1=1
    for k in sizes
        contador2=contador2+k
        for j in contador1:contador2
            for i in j+1:n
                rand1=rand()
                rand2=rand()
                if(i>=contador1 && i<=contador2)
                    if(rand1<=probabilities[proba])
                        A[i,j]=1
                    end
                else
                    if(rand2<=q)
                        A[i,j]=1
                    end
                end
            end
        end
        proba=proba+1
        contador1=contador1+k
    end
    return(A+A')
end

erdosgraph2 (generic function with 1 method)

In [4]:
#Erdos graph function that fixes an r,s entry of the matrix to 1. This allows to test if 
# our method vs the mean method. 

function erdosgraph3(probabilities,q,sizes,n,r,s)
    A= zeros(n,n)
    contador2=0
    proba = 1
    contador1=1
    for k in sizes
        contador2=contador2+k
        for j in contador1:contador2
            for i in j+1:n
                rand1=rand()
                rand2=rand()
                if(i>=contador1 && i<=contador2)
                    if(rand1<=probabilities[proba])
                        A[i,j]=1
                    end
                else
                    if(rand2<=q)
                        A[i,j]=1
                    end
                end
            end
        end
        proba=proba+1
        contador1=contador1+k
    end
    adjacencia = A+A'
    adjacencia[r,s] = 1
    adjacencia[s,r] = 1

    return(adjacencia)
end


erdosgraph3 (generic function with 1 method)

In [51]:
#method for matrix recovery using the mean of the observations. 
#B the observations to recover the graph
#n dimension of the adjacency matrices
#N number of observations

function meanGraphLearn(B,n,N)
    A= sum(B)/N
    return A
end
    


meanGraphLearn (generic function with 1 method)

In [1]:
#b1=Array{Any}(2)
#[b1[r]=erdosgraph([0.8,1,1],0.1,[5,3,4],12) for r in 1:2]
#respuesta=learnGraph([1/2,1/2],0.3,0.1,12,2)

In [23]:
respuesta=learnError([0.5,0.5],0.1,real,papa1,12,2)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1587            
  Cones                  : 0               
  Scalar variables       : 681             
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 276
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective 

3-element Array{Array,1}:
 Array{Float64,2}[[1.68935e-9 8.06286e-8 … -8.30729e-10 -8.37696e-10; 8.06286e-8 1.63678e-9 … -8.5831e-10 -8.41808e-10; … ; -8.30729e-10 -8.5831e-10 … 1.63679e-9 6.79016e-8; -8.37696e-10 -8.41808e-10 … 6.79016e-8 1.67261e-9], [1.68935e-9 8.06286e-8 … -8.30729e-10 -8.37696e-10; 8.06286e-8 1.63678e-9 … -8.5831e-10 -8.41808e-10; … ; -8.30729e-10 -8.5831e-10 … 1.63679e-9 6.79016e-8; -8.37696e-10 -8.41808e-10 … 6.79016e-8 1.67261e-9]]
 [-1.0 1.0 … -1.0 -1.0; 1.0 -1.0 … -1.0 -1.0; … ; -1.0 -1.0 … -1.0 1.0; -1.0 -1.0 … 1.0 -1.0]                                                                                                                                                                                                                                                                                                                                                    
 [-9.628e-9, -1.59974e-8]                                                                                 

In [20]:
papa1 = Array{Any}(2)
papa1[1]=erdosgraph([0.8,1,1],0.1,[5,3,4],12)
papa1[2]=erdosgraph([0.8,1,1],0.1,[5,3,4],12)
real = erdosgraph([1,1,1],0,[5,3,4],12)
respuesta=learnError([0.5,0.5],0.1,real,papa1,12,2)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1587            
  Cones                  : 0               
  Scalar variables       : 681             
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 276
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective 

3-element Array{Any,1}:
 33.9681                                                                                       
   [-1.0 1.0 … -1.0 -1.0; 1.0 -1.0 … -1.0 -1.0; … ; -1.0 -1.0 … -1.0 1.0; -1.0 -1.0 … 1.0 -1.0]
   [-9.628e-9, -1.59974e-8]                                                                    

In [21]:
respuesta[1]

33.968094811088754

In [10]:
respuesta[2]

12×12 Array{Float64,2}:
 -1.0   1.0   1.0   1.0   1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
  1.0  -1.0   1.0   1.0   1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
  1.0   1.0  -1.0   1.0   1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
  1.0   1.0   1.0  -1.0   1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
  1.0   1.0   1.0   1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0   1.0   1.0  -1.0  -1.0  -1.0  -1.0
 -1.0  -1.0  -1.0  -1.0  -1.0   1.0  -1.0   1.0  -1.0  -1.0  -1.0  -1.0
 -1.0  -1.0  -1.0  -1.0  -1.0   1.0   1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0   1.0   1.0   1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0   1.0  -1.0   1.0   1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0   1.0   1.0  -1.0   1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0   1.0   1.0   1.0  -1.0

In [11]:
respuesta[3]

1-element Array{Float64,1}:
 -2.1837e-7